**Loading Datasets from file system**

In [0]:

import numpy as np
import pandas as pd 
import os
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,rmsprop
from random import randrange

#filtering out different book categories
class_names=[]
for root, dirname, filenames in os.walk('book-covers'):
    for className in dirname:
        class_names.append(className)
#print(class_names)

class_names=class_names[0:5]
print(class_names)

# Mappong between category->List of Files in that category
filesAndClassMap={}
for class_name in class_names:
    for _,_,filenames in os.walk('book-covers/'+class_name):
        filesAndClassMap[class_name]=filenames;


#Loading Image Files as arrays of shape 224*224*3
def load_image(filename):
    img = load_img(filename, target_size=(224, 224))
    img = img_to_array(img)
    img = img.reshape(1, 224, 224, 3)
    img = img.astype('float32')
    img = img / 255.0
    return img

#To check if a list is empty or not
def isEmpty(collection)
    if not collection:
              return True
    else:
              return False
train_images=[]
train_labels=[]

#Loading Images from Random classes to generate a dataset file including all book covers and label file with corresponding labels
def load_images():
    while(len(class_names)!=0):
        random_index = randrange(len(class_names))
        random_className = class_names[random_index]
        if not isEmpty(filesAndClassMap[random_className]):
            file=filesAndClassMap[random_className][0]
            if(file.split(".")[1]=="csv"):
                filesAndClassMap[random_className].pop(0)
            if(file.split(".")[1]=="jpg"):
                train_images.extend(load_image('book-covers/'+random_className+"/"+file))
                train_labels.append(random_index)
                filesAndClassMap[random_className].pop(0)
        if isEmpty(filesAndClassMap[random_className]):
            class_names.pop(random_index)
            print("No Of Classes to Process="+str(len(class_names)))
        
load_images()

print(len(train_images))
np.savez('dataset',train_images)
np.savez('labels',train_labels)
 


**Training CNN Model using the Loaded Dataset**

In [0]:
import numpy as np
import pandas as pd 
import os
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,rmsprop
from random import randrange
from datetime import datetime
from packaging import version
from keras import regularizers

#Decompressing and loading dataset and labels
images_compressed=np.load('dataset.npz')
labels_compressed=np.load('labels.npz')
images=images_compressed.f.arr_0
labels=labels_compressed.f.arr_0

#Fetching categories
class_names=[]
for root, dirname, filenames in os.walk('book-covers'):
    for className in dirname:
        class_names.append(className)

class_names=class_names[0:5]
print(class_names)

#Plotting first 25 images for checking the images and labels
def plot_images(images,labels):
    pyplot.figure(figsize=(10,10))
    for i in range(25):
        pyplot.subplot(5,5,i+1)
        pyplot.xticks([])
        pyplot.yticks([])
        pyplot.grid(False)
        pyplot.imshow(images[i], cmap=pyplot.cm.binary)    
        pyplot.xlabel(class_names[labels[i]])
    pyplot.show()




#splitting into training and test sets
train_images=images[500:,:,:,:]
train_labels=labels[500:]
test_images=images[0:500,:,:,:]
test_labels=labels[0:500]

plot_images(train_images,train_labels)


#if book belongs to i th class the [0,0....i th index will be 1....0,0,0]
import numpy as np
def output_labels(x):
  output=np.zeros((len(x),5))
  for idx,val in enumerate(x):
    output[idx][val]=1               
  return output

test_labels=output_labels(test_labels)
train_labels=output_labels(train_labels)

# Convolutional neural network setup
def cnn_model():
  model = models.Sequential()
  model.add(tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(5, activation='softmax'))
  opt = tf.keras.optimizers.RMSprop(lr=0.001, decay=1e-6)
  model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
  model.summary()
  return model

#Plotting accuracy and losslossm against epoch
def model_diagonosis(history):
  # plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
 

#Training and saving the model for future use.
def training_model():
  with tf.device('/device:GPU:0'):
    model=cnn_model()
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    datagen = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
    it_train = datagen.flow(train_images,train_labels,batch_size=64)
    history = model.fit(train_images,train_labels,epochs=50, validation_data=(test_images, test_labels), verbose=2,callbacks=[tensorboard_callback])
    _,accuracy=model.evaluate(test_images,test_labels,verbose=2)
    print('> %.3f' % (accuracy * 100.0))
    model_diagonosis(history)
    model.save('computed_model.h5')
  
training_model()


**Mounting Google Drive onto the Colab Platform**

In [0]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

**Loading computed CNN model, providing sample input image and generating the output**

In [0]:
from numpy import loadtxt
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import tensorflow as tf

def load_image(filename):
    img = load_img(filename, target_size=(224, 224))
    img = img_to_array(img)
    img = img.reshape(1, 224, 224, 3)
    img = img.astype('float32')
    img = img / 255.0
    return img
    
test_image = load_image("/content/drive/My Drive/sample images/sample3.jpg")
# load model
model = tf.keras.models.load_model('/content/drive/My Drive/5 category model.h5')
classes = model.predict(test_image)[0]
labels=['Art-Photography', 'Crafts-Hobbies', 'Poetry-Drama', 'Science-Geography', 'Reference']
for i in range(len(labels)):
    print(labels[i]+"---->%2.5f" % (classes[i]*100))



Using TensorFlow backend.


Art-Photography---->0.32102
Crafts-Hobbies---->0.02694
Poetry-Drama---->99.62040
Science-Geography---->0.01661
Reference---->0.01503


In [0]:
# Installing Kaggle.
!pip install kaggle


In [0]:
#setting up kaggle ennvironment.
import json
token = {"username":"*******","key":"********"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
#Kaggle API calls for loading book cover dataset
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d lukaanicin/book-covers-dataset
!unzip book-covers-dataset.zip
